In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CSV Reader").getOrCreate()


In [5]:

df = spark.read.option("header", True).csv("/content/sample_data/california_housing_train.csv")
df.show(5)



+-----------+---------+------------------+-----------+--------------+-----------+----------+-------------+------------------+
|  longitude| latitude|housing_median_age|total_rooms|total_bedrooms| population|households|median_income|median_house_value|
+-----------+---------+------------------+-----------+--------------+-----------+----------+-------------+------------------+
|-114.310000|34.190000|         15.000000|5612.000000|   1283.000000|1015.000000|472.000000|     1.493600|      66900.000000|
|-114.470000|34.400000|         19.000000|7650.000000|   1901.000000|1129.000000|463.000000|     1.820000|      80100.000000|
|-114.560000|33.690000|         17.000000| 720.000000|    174.000000| 333.000000|117.000000|     1.650900|      85700.000000|
|-114.570000|33.640000|         14.000000|1501.000000|    337.000000| 515.000000|226.000000|     3.191700|      73400.000000|
|-114.570000|33.570000|         20.000000|1454.000000|    326.000000| 624.000000|262.000000|     1.925000|      65500.

In [9]:
df.schema

StructType([StructField('longitude', StringType(), True), StructField('latitude', StringType(), True), StructField('housing_median_age', StringType(), True), StructField('total_rooms', StringType(), True), StructField('total_bedrooms', StringType(), True), StructField('population', StringType(), True), StructField('households', StringType(), True), StructField('median_income', StringType(), True), StructField('median_house_value', StringType(), True)])

In [10]:
df.columns                 # list columns


['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income',
 'median_house_value']

In [11]:
df.describe().show()       # summary stats


+-------+-------------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+
|summary|          longitude|          latitude|housing_median_age|      total_rooms|   total_bedrooms|        population|       households|     median_income|median_house_value|
+-------+-------------------+------------------+------------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+
|  count|              17000|             17000|             17000|            17000|            17000|             17000|            17000|             17000|             17000|
|   mean|-119.56210823529375|  35.6252247058827| 28.58935294117647|2643.664411764706|539.4108235294118|1429.5739411764705|501.2219411764706| 3.883578100000021|207300.91235294117|
| stddev| 2.0051664084260357|2.1373397946570867|12.586936981660406|2179.947071452777|421.4994515798648| 1

In [13]:
from pyspark.sql import functions as F


✅ Average house value by median income bracket:




In [14]:
df.groupBy(
    F.round("median_income", 1).alias("income_bracket")
).agg(
    F.avg("median_house_value").alias("avg_house_value")
).orderBy("income_bracket").show()


+--------------+------------------+
|income_bracket|   avg_house_value|
+--------------+------------------+
|           0.5|144905.55555555556|
|           0.6|          111300.0|
|           0.7| 158657.2142857143|
|           0.8|          137950.0|
|           0.9|128138.68181818182|
|           1.0|107740.90909090909|
|           1.1| 102557.8947368421|
|           1.2| 114496.0594059406|
|           1.3|          105275.0|
|           1.4| 98682.78145695364|
|           1.5|108070.85714285714|
|           1.6|106120.26293103448|
|           1.7|105001.96393442623|
|           1.8|108815.83397683398|
|           1.9|115284.70068027212|
|           2.0|122114.09668508287|
|           2.1|126340.62215909091|
|           2.2|126268.87871853547|
|           2.3| 135354.3523573201|
|           2.4| 136875.4770889488|
+--------------+------------------+
only showing top 20 rows



In [15]:
df.orderBy(df["population"].desc()).select(
    "longitude", "latitude", "population", "median_house_value"
).show(5)


+-----------+---------+----------+------------------+
|  longitude| latitude|population|median_house_value|
+-----------+---------+----------+------------------+
|-120.000000|38.930000|999.000000|     313400.000000|
|-121.210000|37.810000|999.000000|     172100.000000|
|-120.420000|38.420000|999.000000|      91900.000000|
|-119.790000|36.550000|999.000000|      76800.000000|
|-120.430000|34.890000|999.000000|     158000.000000|
+-----------+---------+----------+------------------+
only showing top 5 rows



In [ ]:
✅ Top 5 most populated blocks:



In [16]:
df.orderBy(df["population"].desc()).select(
    "longitude", "latitude", "population", "median_house_value"
).show(5)


+-----------+---------+----------+------------------+
|  longitude| latitude|population|median_house_value|
+-----------+---------+----------+------------------+
|-120.000000|38.930000|999.000000|     313400.000000|
|-121.210000|37.810000|999.000000|     172100.000000|
|-120.420000|38.420000|999.000000|      91900.000000|
|-119.790000|36.550000|999.000000|      76800.000000|
|-120.430000|34.890000|999.000000|     158000.000000|
+-----------+---------+----------+------------------+
only showing top 5 rows



✅ Filter houses younger than 10 years old with high median income:




In [17]:
df.filter(
    (df["housing_median_age"] < 10) & (df["median_income"] > 5)
).show()


+-----------+---------+------------------+------------+--------------+-----------+-----------+-------------+------------------+
|  longitude| latitude|housing_median_age| total_rooms|total_bedrooms| population| households|median_income|median_house_value|
+-----------+---------+------------------+------------+--------------+-----------+-----------+-------------+------------------+
|-115.590000|32.790000|          8.000000| 2183.000000|    307.000000|1000.000000| 287.000000|     6.381400|     159900.000000|
|-116.260000|33.650000|          3.000000| 7437.000000|   1222.000000| 574.000000| 302.000000|    10.294800|     382400.000000|
|-116.370000|33.690000|          7.000000| 8806.000000|   1542.000000| 858.000000| 448.000000|     7.800500|     318100.000000|
|-116.400000|33.780000|          8.000000| 3059.000000|    500.000000| 612.000000| 208.000000|     6.872900|     259200.000000|
|-116.440000|33.740000|          5.000000|  846.000000|    249.000000| 117.000000|  67.000000|     7.988

✅ Correlation between median income and house value:



# New section

In [21]:
df = df.withColumn("median_income", df["median_income"].cast("double")) \
       .withColumn("median_house_value", df["median_house_value"].cast("double"))


In [22]:
corr = df.stat.corr("median_income", "median_house_value")
print(f"Correlation: {corr}")



Correlation: 0.6918706037806676


✅ Register as a temp SQL table:



In [23]:
df.createOrReplaceTempView("housing")

spark.sql("""
  SELECT
    ROUND(median_income, 1) as income_bracket,
    AVG(median_house_value) as avg_value
  FROM housing
  GROUP BY income_bracket
  ORDER BY income_bracket
""").show()


+--------------+------------------+
|income_bracket|         avg_value|
+--------------+------------------+
|           0.5|144905.55555555556|
|           0.6|          111300.0|
|           0.7| 158657.2142857143|
|           0.8|          137950.0|
|           0.9|128138.68181818182|
|           1.0|107740.90909090909|
|           1.1| 102557.8947368421|
|           1.2| 114496.0594059406|
|           1.3|          105275.0|
|           1.4| 98682.78145695364|
|           1.5|108070.85714285714|
|           1.6|106120.26293103448|
|           1.7|105001.96393442623|
|           1.8|108815.83397683398|
|           1.9|115284.70068027212|
|           2.0|122114.09668508287|
|           2.1|126340.62215909091|
|           2.2|126268.87871853547|
|           2.3| 135354.3523573201|
|           2.4| 136875.4770889488|
+--------------+------------------+
only showing top 20 rows

